# Softmax Classification

Softmax Classification을 구현해보자!

## Softmax

In [1]:
import torch
import torch.nn.functional as F

from torch import nn, optim

torch.manual_seed(1)

In [2]:
z = torch.FloatTensor([1, 2, 3])

hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


## Cross Entropy Loss

In [3]:
z = torch.rand(3, 5, requires_grad = True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [4]:
# 정답 값을 임의로 생성
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [5]:
# one-hot encoding
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [6]:
loss = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(loss)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [7]:
loss = (y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()
print(loss)

tensor(1.4689, grad_fn=<MeanBackward0>)


### Cross Entropy Loss - Low Level

In [8]:
loss = (y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()
print(loss)

tensor(1.4689, grad_fn=<MeanBackward0>)


### Cross Entropy Loss - High Level

In [9]:
# NLL(Negative Log Likelihood)
loss = F.nll_loss(F.log_softmax(z, dim=1), y)
print(loss)

tensor(1.4689, grad_fn=<NllLossBackward>)


In [10]:
loss = F.cross_entropy(z, y)
print(loss)

tensor(1.4689, grad_fn=<NllLossBackward>)


## Training with Low-level Cross Entropy Loss

In [11]:
x_datas = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_datas = [2, 2, 2, 1, 1, 1, 0, 0]

In [12]:
x_train = torch.FloatTensor(x_datas)
y_train = torch.LongTensor(y_datas)

print(x_train.shape, y_train.shape)

torch.Size([8, 4]) torch.Size([8])


In [13]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)
n_epochs = 1000

for epoch in range(n_epochs+1):
    optimizer.zero_grad()
    
    h = x_train.matmul(W) + b
    y_one_hot = torch.zeros_like(h)
    y_one_hot.scatter_(1, y_train.unsqueeze(dim=1), 1)
    
    loss = (y_one_hot * -torch.log(F.softmax(h, dim=1))).sum(dim=1).mean()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("Epoch: {:3d}/{}, Loss: {:.6f}".format(epoch, n_epochs, loss))

Epoch:   0/1000, Loss: 1.098612
Epoch: 100/1000, Loss: 0.761050
Epoch: 200/1000, Loss: 0.689991
Epoch: 300/1000, Loss: 0.643229
Epoch: 400/1000, Loss: 0.604117
Epoch: 500/1000, Loss: 0.568255
Epoch: 600/1000, Loss: 0.533922
Epoch: 700/1000, Loss: 0.500291
Epoch: 800/1000, Loss: 0.466908
Epoch: 900/1000, Loss: 0.433507
Epoch: 1000/1000, Loss: 0.399962


## Trainging with F.cross_entropy

In [14]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)
n_epochs = 1000

for epoch in range(n_epochs+1):
    optimizer.zero_grad()
    
    h = x_train.matmul(W) + b
    loss = F.cross_entropy(h, y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("Epoch: {:3d}/{}, Loss: {:.6f}".format(epoch, n_epochs, loss))

Epoch:   0/1000, Loss: 1.098612
Epoch: 100/1000, Loss: 0.761050
Epoch: 200/1000, Loss: 0.689991
Epoch: 300/1000, Loss: 0.643229
Epoch: 400/1000, Loss: 0.604117
Epoch: 500/1000, Loss: 0.568255
Epoch: 600/1000, Loss: 0.533922
Epoch: 700/1000, Loss: 0.500291
Epoch: 800/1000, Loss: 0.466908
Epoch: 900/1000, Loss: 0.433506
Epoch: 1000/1000, Loss: 0.399962


## High-level Implementation with nn.Module

In [15]:
class SoftmaxClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)
        
    def forward(self, x):
        return self.linear(x)
    
model = SoftmaxClassifier()

In [16]:
optimizer = optim.SGD(model.parameters(), lr=0.1)
n_epochs = 1000

for epoch in range(n_epochs+1):
    optimizer.zero_grad()
    
    h = model(x_train)
    loss = F.cross_entropy(h, y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("Epoch: {:3d}/{}, Loss: {:.6f}".format(epoch, n_epochs, loss))

Epoch:   0/1000, Loss: 1.849513
Epoch: 100/1000, Loss: 0.689894
Epoch: 200/1000, Loss: 0.609259
Epoch: 300/1000, Loss: 0.551218
Epoch: 400/1000, Loss: 0.500141
Epoch: 500/1000, Loss: 0.451947
Epoch: 600/1000, Loss: 0.405051
Epoch: 700/1000, Loss: 0.358733
Epoch: 800/1000, Loss: 0.312912
Epoch: 900/1000, Loss: 0.269522
Epoch: 1000/1000, Loss: 0.241922
